# CUSTOMER SEGMENTATION WITH RFM ANALYSIS

# Data
This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.


**Variables**
1. **Invoice:** Invoice number, unique identifier variable for each transaction. Refund invoice numbers starts with "C"
2. **StockCode:** Unique product code
3. **Description:** Product name
4. **Quantity:** The number of product in the invoice
5. **InvoiceDate:** Date and time of the purchase
6. **Price:** Unit price of a product (in terms of Sterlin)
7. **CustomerID:** Unique customer identifier
8. **Country:** Residential country of customers

In [1]:
import datetime as dt
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Reading dataset
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [3]:
# Copying the dataset
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [6]:
df.shape

(541910, 8)

In [4]:
# Checking numerical variables
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.000,9.552,218.081,-80995.000,1.000,3.000,10.000,80995.000
Price,541910.000,4.611,96.760,-11062.060,1.250,2.080,4.130,38970.000
Customer ID,406830.000,15287.684,1713.603,12346.000,13953.000,15152.000,16791.000,18287.000


In [5]:
# Checking null variables 
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
# Removing null observations
df.dropna(inplace=True)

In [8]:
# Removing canceled orders
df = df[~df['Invoice'].astype(str).str.contains('C', na=False)]

df = df[(df['Quantity'] > 0)]

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.000,13.022,180.420,1.000,2.000,6.000,12.000,80995.000
Price,397925.000,3.116,22.097,0.000,1.250,1.950,3.750,8142.750
Customer ID,397925.000,15294.309,1713.173,12346.000,13969.000,15159.000,16795.000,18287.000


In [10]:
df = df[df['Price'] > 0]

In [11]:
# Calculating total price per transaction 
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [12]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [13]:
# max(InvoiceDate) + 2 days
today_date = dt.datetime(2011, 12, 11)

# CALCULATING THE RFM METRICS

In [14]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [15]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [16]:
rfm.reset_index(inplace=True)
rfm.head()

,Customer ID,recency,frequency,monetary
0,12346.000,326,1,77183.600
1,12347.000,3,7,4310.000
2,12348.000,76,4,1797.240
3,12349.000,19,1,1757.550
4,12350.000,311,1,334.400


# CALCULATING THE RFM SCORES

In [17]:
rfm['recency_score'] = pd.qcut(rfm['recency'],5,labels=[5,4,3,2,1])
rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method='first'),5,labels=[1,2,3,4,5])
rfm['monetary_score'] = pd.qcut(rfm['monetary'],5,labels=[1,2,3,4,5])

In [18]:
# Concating scores and assign it to new column
rfm['RFM_SCORE'] = (rfm['recency_score'].astype(str) + 
                    rfm['frequency_score'].astype(str))

In [19]:
rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
0,12346.000,326,1,77183.600,1,1,5,11
1,12347.000,3,7,4310.000,5,5,5,55
2,12348.000,76,4,1797.240,2,4,4,24
3,12349.000,19,1,1757.550,4,1,4,41
4,12350.000,311,1,334.400,1,1,2,11


# NAMING RF SCORES and DEFINING CUSTOMER SEGMENT

In [20]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [21]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,12346.000,326,1,77183.600,1,1,5,11,hibernating
1,12347.000,3,7,4310.000,5,5,5,55,champions
2,12348.000,76,4,1797.240,2,4,4,24,at_risk
3,12349.000,19,1,1757.550,4,1,4,41,promising
4,12350.000,311,1,334.400,1,1,2,11,hibernating
